In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools
import unidecode

from filing import Filing

from engineFD import EngineFD
from engineDK import EngineDK
from scraper._dates import PLAYOFF_DATES

In [2]:
pd.options.display.max_rows = 100
plt.style.use('fivethirtyeight') 

In [3]:
def percentile(n: int) -> float:
    """
    Calculates n% outcome for players, designed for use in .agg
    Example: df.groupby('name')['fpts'].agg([percentile(0), percentile(50), percentile(100)])
         --> Returns 3 columns, indexed by name corresponding to following outcomes: 0% (minimum), 50% (median), 100% (maxium)
         --> Common usage will be 25% which roughly corresponds to floor and 75% which roughly corresponds to ceiling
    """
    def percentile_(arr):
        return np.percentile(arr, n)
    # percentile_.__name__ = f'percentile_{n}'
    label = {25: 'floor', 50: 'median', 75: 'ceiling'}.get(n, f'{n}%')
    percentile_.__name__ = label
    return percentile_

def valuerange(arr):
    """
    Returns the range of values, max-min, to demonstrate spread of possible outcomes
    For small samples, more indicative of spread than std which can often be confusing
    """

    valuerange.__name__ = 'range'
    return np.max(arr) - np.min(arr)

In [4]:
class SeasonData:
    def __init__(self, year=2022, **kwargs):
        self.season = f'{year}-{year+1}'
        self.filing = Filing(self.season)

        self.site = kwargs.get('site', 'draftkings')
    
    def load(self, **kwargs):

        if hasattr(self, 'clean'):
            return self.clean
        
        raw = self.filing.load_boxscores()

        columns = [
            'date',
            'name',
            'starter',
            'playoffs', # **
            'team',
            'opp',

            'mp',
            'fpts', # ** Originally has both DraftKings and FanDuel columns and selects one based 
            'fppm', # **
            
            'pts',
            'ast',
            'trb',
            'stl',
            'blk',
            'tov',
            
            'usg',
            'ts',
            'ast_pct',

            'spread',
            'total'

            # ** indicates columns added and not in original data
        ]

        playoffs_start, playoffs_end = PLAYOFF_DATES[self.season]
        playoffs_dates = [ date_.strftime('%Y-%m-%d') for date_ in pd.date_range(playoffs_start, playoffs_end) ]

        raw['playoffs'] = raw['date'].map(lambda date_: int(date_ in playoffs_dates))
        # Clean up name
        raw['name'] = raw['name'].map(lambda name_: unidecode.unidecode(name_))

        self.clean = (raw
                      .assign(
                          fpts=lambda df_: df_.dk_fpts if self.site == 'draftkings' else df_.fd_fpts,
                          fppm=lambda df_: df_.fpts / df_.mp,
                          # mp_usg=lambda df_: df_.mp * df_.usg,
                          **{c: lambda df_, c=c:df_[c].astype('int') for c in ('pts', 'ast', 'trb', 'stl', 'blk', 'tov', 'playoffs')}
                      )
                      .sort_values(['date', 'team', 'fpts'], ascending=[True, True, False])
                      .round(3)
                      [columns]
                     )
        
        return self.clean

    def create_heatmap(self, **kwargs):

        

        # Make sure stats include in load() if want to add more
        stats = sum([
            ['fpts', 'mp', 'usg', 'ts', 'ast_pct', 'spread', 'total'],
            kwargs.get('stats', [])
        ], list())

        # Conditional included to check if just starters or not
        df = (self.load()
              .pipe(lambda df_: df_.loc[df_['starter'].isin(kwargs.get('starter', (0,1)))])
              [stats]
             )

        output = [
            f'Sample size: {df.shape[0]:,}',
            f'Stats included: {", ".join(stats[:-1])} and {stats[-1]}',
        ]

        # Include product of stats
        if 'product' in kwargs:
            product_str = 'Product columns:'
            for cols in itertools.combinations(kwargs['product'], 2):
                df['*'.join(cols)] = df[cols[0]]*df[cols[1]]
                # stats.append(cols)
                product_str += f' {cols}'
            output.append(product_str)

        # Correlations dataframe
        corr: pd.DataFrame = df.corr()
            
        # Better size
        fig, ax = plt.subplots(figsize=kwargs.get('figsize', (15,10)))
        mask: np.ndarray = np.triu(np.ones_like(corr, dtype=bool))
        
        # Preferred kwargs for heatmap that are not defaults
        sns_kwargs: dict[str,str|float] = {
            'cmap': kwargs.get('cmap', 'jet_r'),
            'vmin': kwargs.get('vmin', 0.4),
            'vmax': kwargs.get('vmax', 1.0)
            
        }

        print(*output, sep='\n')
        
        return sns.heatmap(
            corr, 
            mask=mask,
            **sns_kwargs
        ) 

In [5]:
class ContestData():

    def __init__(self, **kwargs) -> None:
        """
        Creates object for contest data, where data is built around provided contest files
        Defaults:
            - Site: DraftKings
            - Mode: Main-Slate
        """

        self.site = kwargs.get('site', 'draftkings')
        self.mode = kwargs.get('mode', 'main-slate')

        self.filing = Filing('2023-2024')
        # using last years data until sample size built up
        self.season = SeasonData(year=2023, site=self.site)

        self.contest_path = os.path.join(self.filing.season_dir, 'contest-files', self.site, self.mode, f'{datetime.date.today().isoformat()}-late.csv')

        columns = {
            'draftkings': {
                'Name': 'name',
                'Position': 'pos',
                'Salary': 'salary',
                'TeamAbbrev': 'team'
            },
            'fanduel': {
                'Nickname': 'name',
                'Position': 'pos',
                'Salary': 'salary',
                'Team': 'team',
                'Opponent': 'opp',
                'Injury Indicator': 'injury'
            }
        }


        # LATE NOW
        self.data = (pd
                     .read_csv(
                         self.contest_path,
                         usecols=columns[self.site]
                     )
                     # .pipe(lambda df_: df_.loc[df_['Injury Indicator'] != 'O'])
                     # .drop('Injury Indicator', axis=1)
                     .rename(columns[self.site], axis=1)
                     .set_index('name')
                    )

        self.data.index = self.data.index.map(lambda name: ' '.join(name.split(' ')[:2]).replace('.', ''))

        # Season data for all players in current contest
        self.szn = (self.season.load()
                    .pipe(lambda df_: df_.loc[df_['name'].isin(self.data.index)])
                   )

    def getIDs(self) -> dict[str,str]:
        """
        Returns dict with key as names and IDs for csv upload as values
        """

        id_columns = ['Id', 'Nickname'] if self.site == 'fanduel' else ['ID', 'Name']

        df = (pd
              .read_csv(self.contest_path, usecols=id_columns)
              [id_columns]
              .set_axis(['id', 'name'], axis=1).set_index('name')
             )

        df.index = df.index.map(lambda name: ' '.join(name.split(' ')[:2]).replace('.', ''))

        return {name: df.loc[name, 'id'] for name in df.index}

    def create_medians(self, **kwargs):
        """
        Loads medians for stats
        """

        if hasattr(self, 'df_medians'):
            return self.df_medians

        stats = sum([
            ['mp', 'fpts', 'fppm', 'usg', 'ts', 'mp*usg', 'mp*usg*ts'],
            kwargs.get('stats', [])
        ], list())
        
        ret = (self.szn
               .assign(
                   usg=lambda df_: df_.usg / 100,
                   mp_usg=lambda df_: df_.mp * df_.usg,
                   mp_usg_ts=lambda df_: df_.mp_usg * df_.ts,
                   fppm_mp=lambda df_: df_.fppm * df_.mp,
               )
               .rename({'mp_usg': 'mp*usg', 'fppm_mp': 'fppm*mp', 'mp_usg_ts': 'mp*usg*ts'}, axis=1)
               .groupby('name')
               [stats]
               .agg(['count', percentile(25), 'median', percentile(75)])
               .drop([(stat_, 'count') for stat_ in stats[1:]], axis=1)
               .rename({('mp', 'count'): ('mp', 'n-games')}, axis=1)
              )

        self.df_medians = (ret
                           .set_axis(sum([
                               ['n-games'],
                               ['-'.join(col) for col in ret.columns][1:]
                           ], list()),axis=1)
                           .sort_values(['mp*usg-median', 'fppm-median', 'usg-median'], ascending=False)
                           .round(3)
                          )

        return self.df_medians

    def load(self, **kwargs) -> pd.DataFrame:

        inactive = kwargs.get('inactive', [])
        if len(inactive): self.data = self.data.loc[self.data.index.isin(inactive) == False]
        # medians = self.create_medians()
        # self.data['fpts'] = self.data.index.map(lambda name: medians.loc[name, 'fpts-median'] if name in medians.index else 0.0)

        source = kwargs.get('source', 'props')


        if source == 'medians':
            medians = self.create_medians(**kwargs)
            for col in ('mp', 'fpts', 'fppm', 'usg', 'mp*usg', 'mp*usg*ts') + tuple(kwargs.get('stats', list())):
                self.data[col] = self.data.index.map(lambda name: medians.loc[name, f'{col}-median'] if name in medians.index else 0.0)

            self.data = (self.data
                         # .loc[self.data['mp'] > 8.0]
                         .assign(fpts_1k=lambda df_: 1_000 * df_.fpts / df_.salary)
                        )

        if source == 'props':
            projection_path = os.path.join(self.filing.season_dir, 'contest-files', self.site, 'current', f'projections.csv')
            projections = (pd
                           .read_csv(projection_path)
                           .set_index('name')
                          )

            for projection_column in ('fpts', 'e_fpts', 'fpts/$', 'e_fpts/$', 'value'):
                self.data[projection_column] = self.data.index.map(lambda name: projections.loc[name, projection_column] if name in projections.index else 0.0)

        self.data['salary'] = self.data['salary'].astype('int')

        edits = {
            # 'Khris Middleton': 22.0,
            # 'Quentin Grimes': 9.0,
            # 'Cameron Payne': 9.0,
            # 'Cam Reddish': 12.0,
            # 'Mitchell Robinson'

        }

        for name, edit in edits.items():
            self.data.loc[name, 'fpts'] = edit
            self.data.loc[name, 'value'] = 0.0

        return self.data.round(3)

In [6]:
contest = ContestData(site='draftkings')

In [7]:
contest.load(source='medians'); #.pipe(lambda df_: df_.loc[df_.index.isin(['Jabari Smith'])])

In [20]:
avoid_teams = list()

In [21]:
inactive = []

In [8]:
contest_df = (contest.load()
              # .pipe(lambda df_: df_.loc[(df_['fpts'] > 0.0) & (df_['team'].isin(avoid_teams) == False)])
              .pipe(lambda df_: df_.loc[(df_['fpts'] > 0.0)])
              # .pipe(lambda df_: df_.loc[df_.index.isin(inactive) == False])
              # .pipe(lambda df_: df_.loc[df_['team'].isin(['ORL', 'WAS']) == False])
              # .sort_values('mp*usg*ts', ascending=False)
              .sort_values('value', ascending=False)
              .head(50)
             )

# contest_df = (pd
#               .concat([
#                   contest_df, #.head(31),
#                   contest.data.loc[contest.data.index.isin(['Jaime Jaquez', 'Caleb Martin'])].sort_values('e_fpts/$', ascending=False)
#               ])
#              )

In [9]:
contest_df #.sort_values('mp*usg*ts', ascending=False).head(50)

,pos,salary,team,mp,fpts,fppm,usg,mp*usg,mp*usg*ts,fpts_1k,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,,,,,,,
Nikola Jokic,C,11000,DEN,35.233,61.88,1.703,0.259,9.397,6.559,5.455,30.23,5.63,2.75,6.88
Scottie Barnes,PG/SF,8700,TOR,36.033,47.62,1.259,0.244,8.478,4.966,5.259,23.37,5.47,2.69,4.12
Ivica Zubac,C,6100,LAC,26.650,34.37,1.034,0.178,4.499,3.055,4.488,16.20,5.63,2.66,3.87
Jamal Murray,PG,8000,DEN,31.866,42.88,1.280,0.280,8.744,4.912,4.844,22.22,5.36,2.78,2.88
RJ Barrett,SG/SF,6400,TOR,29.983,32.88,0.977,0.274,7.833,4.363,4.453,17.37,5.14,2.71,0.88
Immanuel Quickley,PG/SG,7000,TOR,25.067,35.12,1.034,0.230,5.778,3.605,3.786,18.59,5.02,2.66,0.12
Paul George,SG/SF,8200,LAC,36.534,39.88,1.180,0.276,9.291,5.778,5.168,20.52,4.86,2.50,-1.12
Collin Sexton,PG/SG,6000,UTA,22.467,28.62,1.132,0.276,5.775,3.596,4.062,14.31,4.77,2.38,-1.38
Jordan Clarkson,SG,6200,UTA,30.691,29.12,1.071,0.272,8.017,4.562,5.585,15.62,4.70,2.52,-1.88


In [10]:
print(f'Player pool size: {contest_df.shape[0]}')

Player pool size: 29


In [11]:
pd.options.display.max_rows = 100

In [13]:
engine = EngineDK(
    contest_df, 
    sum_cols=['e_fpts'], #, 'mp*usg', 'fppm', 'mp*usg*ts'],
    past=False
)

In [14]:
import time
def timed_lineups() -> pd.DataFrame:
    """
    Creates lineups while also timing how long it takes to create the lineups
    """


    start = time.perf_counter()
    lineups = (engine.create_lineups()
               .sort_values('fpts', ascending=False)
               .reset_index(drop=True)
              )
    stop = time.perf_counter()
    
    elapsed: float = (stop - start)/60.0
    
    elapsed_str: str = str(elapsed)
    minutes: int = int( elapsed_str.split('.')[0] )
    
    decimals: float = float( f'0.{elapsed_str.split(".")[1]}' )
    seconds: int = int(decimals * 60.0)
    
    performance_time: str = f'{minutes}m {seconds}s.'
    
    print(f'Performance time for creating lineups with {contest_df.shape[0]} players in pool: {performance_time}\n')
    
    return lineups

In [ ]:
# lineups = engine.create_lineups().sort_values('fpts', ascending=False).reset_index(drop=True)
lineups = timed_lineups()

In [ ]:
lineups.head(20)
# doing pg-sg-sf = 2m 41s
# doing pg-sg, sf-pf = 3m 25s

In [15]:
from collections.abc import Sequence

def flatten(nestedSeq: Sequence[Sequence[str,...], ...], **kwargs) -> list[str,...]:
        """
        Takes 2d sequence and returns all values in 1d
        Example: [(a,b,c), (a,y,z), (a,b,z)] -> [a, b, c, a, y, z, a, b, z]
        TODO: kwargs to add functinoality
            - unique: [(a,b,c), (a,y,z), (a,b,z)] -> [a, b, c, y, z]
            - counts dict -> {a: 3, b: 2, z: 2, c: 1, z: 1} (recursive)
            - etc
        """
        if kwargs.get('unique', False):
            return set(cls.flatten(nestedSeq))

        return [element for innerSeq in nestedSeq for element in innerSeq]

def exposures(lineups: tuple[tuple[str,...], ...]) -> pd.Series:
    """
    Takes tuple of lineups and returns overall exposure
    """

    numlineups = len(lineups)
    all_names = flatten(lineups)
    exposure = {name: 100 * all_names.count(name) / numlineups for name in set(all_names)}

    return (pd
            .Series(exposure)
            .sort_values(ascending=True)
            .plot
            .barh(figsize=(15,10))
           )

In [16]:
def create_upload_csv(upl_df: pd.DataFrame):
    df = upl_df.copy(deep=True)

    if 'lineup' not in df.columns:
        df['lineup'] = df[engine.labels].apply(tuple, axis=1)
        df['lineup'] = df['lineup'].map(lambda names: tuple(set(names)))
    
    for name, id in contest.getIDs().items():
        df = df.replace(name, id)

    upload_path = os.path.join(contest.filing.season_dir, 'contest-files', contest.site, 'current', 'IDs.csv')
    # df.to_csv('upload.csv', index=False)
    df.to_csv(upload_path, index=False)

    return exposures(tuple(df['lineup']))

In [17]:
# create_upload_csv(upl);

In [18]:
lineups['lineup'] = lineups[engine.labels].apply(tuple, axis=1)
lineups['lineup'] = lineups['lineup'].map(lambda names: tuple(set(names)))

In [ ]:
top_players = ['Anthony Davis', 'LeBron James', 'Jalen Brunson', 'Kevin Durant', 'Devin Booker', 'Giannis Antetokounmpo', 'Julius Randle', 'Damian Lillard']
top_combos = [tuple(combo) for combo in itertools.combinations(top_players, 2)]

In [28]:
for name in ['Jaylen Brown', 'Jayson Tatum', 'Al Horford', 'Jusuf Nurkic', 'Drew Eubanks', 'Toumani Camara', 'Devin Booker']: # + top_players
    column_ = name.split(' ')[1].lower()
    if column_ not in lineups.columns:
        lineups[column_] = lineups['lineup'].map(lambda names: int(name in names))

In [24]:
lineups['salaries'] = lineups['lineup'].map(lambda names: tuple(sorted([engine.checker.pvalue(name, 'salary') for name in names])))
# lineups['3k'] = lineups['salaries'].map(lambda sals: int(min(sals) < 4_000))
# for n in (1,2,3):
#     lineups[f'{n}-3k'] = lineups['salaries'].map(lambda sals: int(len([sal for sal in sals if sal < 4_000]) == n))

In [19]:
lineups['teams'] = lineups['lineup'].map(lambda names: '-'.join([engine.checker.pvalue(name, 'team') for name in names]))
lineups['distro'] = lineups['teams'].map(lambda teams: tuple(sorted([teams.split('-').count(team) for team in set(teams.split('-'))])))
# lineups['games'] = lineups['lineup'].map(lambda names: '_'.join(engine.checker.games(names)))
# lineups['n_games'] = lineups['lineup'].map(lambda names: len(set(engine.checker.games(names))))
# lineups['n_teams'] = lineups['distro'].map(lambda distro: len(distro))

In [20]:
lineups = lineups.drop([col for col in ['lineup', 'salaries', 'teams', 'games'] if col in lineups.columns], axis=1)

In [21]:
# lineups.sort_values('fppm', ascending=False).head(10)
lineups.sort_values('fpts', ascending=False).head(5)

,PG,SG,SF,PF,C,G,F,UTIL,fpts,salary,e_fpts,distro
0,Marcus Smart,Desmond Bane,Derrick Jones,Grant Williams,Dwight Powell,RJ Barrett,Taurean Prince,Anthony Davis,250.99,50000,129.35,"(1, 2, 2, 3)"
1,Marcus Smart,Desmond Bane,Derrick Jones,Anthony Davis,Dwight Powell,Cam Reddish,Grant Williams,Immanuel Quickley,250.75,49800,127.76,"(1, 2, 2, 3)"
2,Marcus Smart,Desmond Bane,Derrick Jones,Anthony Davis,Grant Williams,Cam Reddish,Josh Green,Immanuel Quickley,250.00,49900,127.82,"(1, 2, 2, 3)"
3,Marcus Smart,Desmond Bane,Derrick Jones,Josh Green,Dwight Powell,RJ Barrett,Grant Williams,Anthony Davis,249.99,49800,128.46,"(1, 1, 2, 4)"
4,Marcus Smart,Desmond Bane,Derrick Jones,Anthony Davis,Dwight Powell,Cam Reddish,Josh Green,RJ Barrett,249.73,50000,128.48,"(1, 2, 2, 3)"


In [30]:

(lineups
 .loc[
 (lineups['tatum'] == 1)
 # (lineups['distro'] == (1,1,2,2,3))
 & (lineups['horford'] == 0)
 & (lineups['brown'] == 1)
 # & (lineups['booker'] == 1)
 & (lineups['nurkic'] == 1)
 & (lineups['eubanks'] == 0)
 & (lineups['camara'] == 0)
 # & (lineups['gilgeous-alexander'] == 1)
 # & (lineups['antetokounmpo'] == 1)
 # & (lineups['lopez'] == 1)
 ]
 # .sort_values(['mp*usg*ts'], ascending=[False])
 .head(20)
 # .index
)

,PG,SG,SF,PF,C,G,F,UTIL,lineup,fpts,salary,e_fpts,brown,tatum,horford,nurkic,eubanks,camara,booker
591,Derrick White,Jaylen Brown,Jayson Tatum,Nassir Little,Dario Saric,Payton Pritchard,Matisse Thybulle,Jusuf Nurkic,"(Derrick White, Jaylen Brown, Jusuf Nurkic, Ma...",235.02,50000,119.36,1,1,0,1,0,0,0
688,Payton Pritchard,Jaylen Brown,Jayson Tatum,Nassir Little,Jusuf Nurkic,Jordan Goodwin,Sam Hauser,Deandre Ayton,"(Jaylen Brown, Jordan Goodwin, Jusuf Nurkic, S...",234.77,49900,119.67,1,1,0,1,0,0,0
924,Payton Pritchard,Jaylen Brown,Jayson Tatum,Jonathan Kuminga,Jusuf Nurkic,Grayson Allen,Nassir Little,Kevon Looney,"(Jonathan Kuminga, Jaylen Brown, Jusuf Nurkic,...",234.27,49800,120.02,1,1,0,1,0,0,0
929,Payton Pritchard,Jaylen Brown,Jayson Tatum,Nassir Little,Jusuf Nurkic,Grayson Allen,Matisse Thybulle,Deandre Ayton,"(Jaylen Brown, Jusuf Nurkic, Grayson Allen, Ma...",234.27,49800,118.40,1,1,0,1,0,0,0
1048,Chris Paul,Jaylen Brown,Jayson Tatum,Nassir Little,Jusuf Nurkic,Payton Pritchard,Sam Hauser,Kevon Looney,"(Jaylen Brown, Jusuf Nurkic, Sam Hauser, Kevon...",234.03,50000,120.31,1,1,0,1,0,0,0
1061,Payton Pritchard,Jaylen Brown,Jayson Tatum,Dario Saric,Jusuf Nurkic,Grayson Allen,Nassir Little,Jonathan Kuminga,"(Jonathan Kuminga, Jaylen Brown, Jusuf Nurkic,...",234.02,49700,119.16,1,1,0,1,0,0,0
1072,Jrue Holiday,Jaylen Brown,Jonathan Kuminga,Nassir Little,Jusuf Nurkic,Payton Pritchard,Matisse Thybulle,Jayson Tatum,"(Jrue Holiday, Jaylen Brown, Jusuf Nurkic, Pay...",234.02,50000,118.92,1,1,0,1,0,0,0
1221,Payton Pritchard,Jaylen Brown,Jayson Tatum,Dario Saric,Jusuf Nurkic,Jordan Goodwin,Nassir Little,Chris Paul,"(Jaylen Brown, Jordan Goodwin, Jusuf Nurkic, N...",233.78,49900,119.51,1,1,0,1,0,0,0
1223,Chris Paul,Jaylen Brown,Jayson Tatum,Nassir Little,Jusuf Nurkic,Payton Pritchard,Sam Hauser,Dario Saric,"(Jaylen Brown, Jusuf Nurkic, Sam Hauser, Nassi...",233.78,49900,119.45,1,1,0,1,0,0,0
1280,Payton Pritchard,Jaylen Brown,Jayson Tatum,Jonathan Kuminga,Jusuf Nurkic,Scoot Henderson,Nassir Little,Grayson Allen,"(Jonathan Kuminga, Scoot Henderson, Jaylen Bro...",233.76,49900,118.88,1,1,0,1,0,0,0


In [30]:
import random

In [35]:
indices = list()

for p1, p2 in top_combos:


    robinson_val = random.randint(0,1)
    quickley_val = random.randint(0,1)
    # grimes_val = random.randint(0,3)
    # payne_val = random.randint(0,3)
    # reddish_val = random.randint(0,2)
    
    p1_last_name, p2_last_name = [name_.split(' ')[1].lower() for name_ in (p1,p2)]

    try:
        idx = list(lineups
                   .loc[
                   (lineups[p1_last_name] == 1)
                   & (lineups[p2_last_name] == 1)
                   & (lineups['robinson'] == robinson_val)
                   & (lineups['quickley'] == quickley_val)
                   # & (lineups['payne'] == payne_val)
                   # & (lineups['reddish'] == reddish_val)
                   ]
                   .head(1)
                   .index
                  )[0]
        
        indices.append(idx)
    except IndexError:
        pass
    
    

indices[:20]

[310,
 62,
 993,
 661,
 14817,
 1368,
 545,
 5,
 89,
 2813,
 10654,
 130,
 475,
 1485,
 1304,
 5112,
 637,
 420,
 2140,
 17362]

In [86]:
len(top_combos)

28